In [120]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [121]:
tf.__version__

'2.13.0'

In [122]:
dataset=pd.read_csv('churn_Modelling.csv')
X=dataset.iloc[:,3:-1].values
y=dataset.iloc[:,-1].values

In [123]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [124]:
print(y)

[1 0 1 ... 1 1 0]


In [125]:
##Encodeing Categorical data

In [126]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [127]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [128]:
##One Hot Encoding the "Geography" column

In [129]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [130]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [131]:
##Splitting the dataset into the Training set and Test set

In [132]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [133]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test = sc.transform(X_test)

 # Buildind ANN

Initializing the ANN 

In [134]:
ann = tf.keras.models.Sequential()

Adding the input layer and first hidden layer

In [135]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding second hiddden layer

In [136]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Adding the output layer

In [137]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Training ANN

compiling ANN with optimizer and loss function

In [138]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the ANN model on the Training set

Training the ANN on the Training set

In [139]:
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 0.7959
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4799 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4515 - accuracy: 0.7983
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4335 - accuracy: 0.8084
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4201 - accuracy: 0.8139
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4092 - accuracy: 0.8226
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3991 - accuracy: 0.8328
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3905 - accuracy: 0.8393
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3830 - accuracy: 0.8425
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3772 - accura

# Part 4 - Making the predictions and evaluating the model

Predicting the result of a single observation

Using our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 60 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer ?

In [152]:
result=ann.predict(sc.transform([[1, 0, 0, 600, 1, 60, 3, 60000, 2, 1, 1, 50000]]))
if result >0.5:
    print('customer is going to say goodbye to bank')
else:
        print('Customer stays in bank')

1/1 [==============================] - 0s 30ms/step
Customer stays in bank


Therefore, our ANN model predicts that this customer stays in the bank!

Important note 1: Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

Important note 2: Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

# Predicting the Test set results

In [154]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
# 0 is indicating he stay in banks
# 1 is indicating he saying goodbye to bank

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


# Making the Confusion Matrix

In [155]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1517   78]
 [ 197  208]]


0.8625

In [156]:
#we have 1517 correct prediction that customer stays in bank.
#we have 208 correct prediction that customer leaves bank.
#we have 78 incorrect predictions that customer leaves the bank.
#we have 197 incorrect prediction that customer stays in bank.